In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

# just for the sake of this blog post!
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
data = pd.read_csv('dengue_features_train.csv', index_col=[0,1,2]) 
labels = pd.read_csv('dengue_labels_train.csv', index_col=[0,1,2])
data = data.join(labels)
data.drop('week_start_date', axis=1, inplace=True)

In [4]:
data.describe()

,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,...,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm,total_cases
count,1262.000000,1404.000000,1434.000000,1434.000000,1443.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,...,1446.000000,1443.000000,1446.000000,1446.000000,1413.000000,1413.000000,1436.000000,1442.000000,1434.000000,1456.000000
mean,0.142294,0.130553,0.203783,0.202305,45.760388,298.701852,299.225578,295.246356,303.427109,295.719156,...,82.161959,45.760388,16.746427,4.903754,27.185783,8.059328,32.452437,22.102150,39.326360,24.675137
std,0.140531,0.119999,0.073860,0.083903,43.715537,1.362420,1.261715,1.527810,3.234601,2.565364,...,7.153897,43.715537,1.542494,3.546445,1.292347,2.128568,1.959318,1.574066,47.455314,43.596000
min,-0.406250,-0.456100,-0.015533,-0.063457,0.000000,294.635714,294.892857,289.642857,297.800000,286.900000,...,57.787143,0.000000,11.715714,1.357143,21.400000,4.528571,26.700000,14.700000,0.000000,0.000000
25%,0.044950,0.049217,0.155087,0.144209,9.800000,297.658929,298.257143,294.118929,301.000000,293.900000,...,77.177143,9.800000,15.557143,2.328571,26.300000,6.514286,31.100000,21.100000,8.700000,5.000000
50%,0.128817,0.121429,0.196050,0.189450,38.340000,298.646429,299.289286,295.640714,302.400000,296.200000,...,80.301429,38.340000,17.087143,2.857143,27.414286,7.300000,32.800000,22.200000,23.850000,12.000000
75%,0.248483,0.216600,0.248846,0.246982,70.235000,299.833571,300.207143,296.460000,305.500000,297.900000,...,86.357857,70.235000,17.978214,7.625000,28.157143,9.566667,33.900000,23.300000,53.900000,28.000000
max,0.508357,0.454429,0.538314,0.546017,390.600000,302.200000,302.928571,298.450000,314.000000,299.900000,...,98.610000,390.600000,20.461429,16.028571,30.800000,15.800000,42.200000,25.600000,543.300000,461.000000


In [5]:
data_sj, data_iq = data.loc['sj'], data.loc['iq']

Handling NaN

In [6]:
data.fillna(0, inplace=True)

In [47]:
from sklearn.model_selection import TimeSeriesSplit


X, y = data_sj.drop('total_cases', axis=1), data_sj['total_cases']


'''X_cv, y_cv = [], []
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(data_sj):
    print("TRAIN:", (train_index[0], train_index[-1]), "TEST:", (test_index[0], test_index[-1]))
    X_cv += [[X.iloc[train_index], X.iloc[test_index]]]
    y_cv += [[y.iloc[train_index], y.iloc[test_index]]]'''

'X_cv, y_cv = [], []\ntscv = TimeSeriesSplit()\nfor train_index, test_index in tscv.split(data_sj):\n    print("TRAIN:", (train_index[0], train_index[-1]), "TEST:", (test_index[0], test_index[-1]))\n    X_cv += [[X.iloc[train_index], X.iloc[test_index]]]\n    y_cv += [[y.iloc[train_index], y.iloc[test_index]]]'

In [10]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()
scaler.fit(X_train)
X_train, X_test = scaler.transform(X_train), scaler.transform(X_test)

In [50]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb


estimator = xgb.XGBClassifier(
    objective= 'binary:logistic',
    n_estimators=1000,
    nthread=4,
    seed=42
)



parameters = {
    'max_depth': range (2, 10, 1),
    'learning_rate': [0.1, 0.01, 0.05]
}


grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    scoring = 'neg_mean_squared_error',
    n_jobs = -1,
    cv = 5,
    verbose=4
)

In [ ]:
grid_search.fit(X, y)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
